In [79]:
import json
import numpy as np

In [80]:
system = 'spring-security'

In [81]:
pull_file_path = f'data/metrics/{system}_info_devs.json'
refactorings_file_path = f'data/metrics/refactorings_{system}.json'
metrics = f'data/metrics/metrics_output.json'

with open(pull_file_path) as json_input:
    dev_info = json.load(json_input)

with open(refactorings_file_path) as json_input:
    refactorings_info = json.load(json_input)

with open(metrics) as json_input:
    metrics = json.load(json_input)

In [82]:
list_participates = []
list_commited = []
list_opened = []
list_reviewed = []
list_commented = []

for current_dev in dev_info:
    dev = dev_info[current_dev]
    
    if dev['participates_all_nfrs_high']:
        list_participates.append(current_dev)
    if dev['commited_all_nfrs_high']:
        list_commited.append(current_dev)
    if dev['opened_discussion_all_nfrs_high']:
        list_opened.append(current_dev)
    if dev['commented_all_nfrs_high']:
        list_commented.append(current_dev)
    if dev['reviewed_all_nfrs_high']:
       list_reviewed.append(current_dev)
    

In [83]:
print ("list_participates")
print (list_participates)

list_participates
['jgrandja', 'rwinch', 'jzheaux', 'pivotal-issuemaster']


In [84]:
print ("list_commited")
print (list_commited)

list_commited
['', 'jgrandja', 'rwinch', 'jzheaux', 'eleftherias']


In [85]:
print ("list_opened")
print (list_commited)

list_opened
['', 'jgrandja', 'rwinch', 'jzheaux', 'eleftherias']


In [86]:
print ("list_reviewed")
print (list_reviewed)

list_reviewed
['jgrandja', 'rwinch', 'jzheaux', 'eleftherias', 'evgeniycheban', 'ch4mpy', 'Buzzardo']


In [87]:
print ("list_commented")
print (list_commented)

list_commented
['jgrandja', 'rwinch', 'jzheaux', 'pivotal-cla', 'eleftherias', 'pivotal-issuemaster']


jgrandja: Commites, Opens, Reviews, Comments
rwinch: Commites, Opens, Reviews, Comments
jzheaux: Commites, Opens, Reviews, Comments

eleftherias: Commites, Opens, Reviews

evgeniycheban: Reviews
ch4mpy: Reviews
Buzzardo: Reviews

pivotal-issuemaster: Comments
pivotal-cla: Comments

In [88]:
for dev_name in refactorings_info:
    dev = refactorings_info[dev_name]
    if dev['num_refactorings'] > quartiles["3rd"]:
        print (dev_name)

rwinch
jzheaux
jgrandja
nor-ek
fhanik
larsgrefer
philwebb
izeye
wkorando
tekul


In [89]:
metrics_spring_security = metrics['spring-security']
users_metrics = metrics_spring_security['user_metrics']

In [90]:
metrics = ['number_of_merged_prs', 'experience_in_days', 'avg_size_of_commits', 'mean_time_between_merged_prs', 
           'number_of_pull_requests_opened', 'pulls_opened', 'number_of_commits', 'pulls_closed', 'mean_discussion_duration',
          'mean_time_between_comments', 'mean_words', 'total_words', 'number_of_comments', 'lines_revised', 'number_files_revised',
          'number_modules_revised', 'number_reviews']

dict_metrics = {}

for user in users_metrics:
    for metric in metrics:
        if metric in user:
            if metric in dict_metrics:
                dict_metrics[metric].append(user[metric])
            else:
                dict_metrics[metric] = []
                dict_metrics[metric].append(user[metric])

quartiles_metrics = {}
for metric in dict_metrics:
    quartiles_metrics[metric] = {}
    
    quartiles_metrics[metric]["1st"] = np.percentile(list(set(dict_metrics[metric])), 25)
    quartiles_metrics[metric]["median"] = np.percentile(list(set(dict_metrics[metric])), 50)
    quartiles_metrics[metric]["3rd"] = np.percentile(list(set(dict_metrics[metric])), 75)

In [91]:
all_refs = []
for dev_name in refactorings_info:
    dev = refactorings_info[dev_name]
    all_refs.append(dev['num_refactorings'])

quartiles_metrics['refactorings'] = {}
quartiles_metrics['refactorings']["1st"] = np.percentile(list(set(all_refs)), 25)
quartiles_metrics['refactorings']["median"] = np.percentile(list(set(all_refs)), 50)
quartiles_metrics['refactorings']["3rd"] = np.percentile(list(set(all_refs)), 75)

In [92]:
json_formatted_str = json.dumps(quartiles_metrics, indent=2)

print(json_formatted_str)

{
  "number_of_merged_prs": {
    "1st": 8.5,
    "median": 31.0,
    "3rd": 93.0
  },
  "experience_in_days": {
    "1st": 763.75,
    "median": 1307.0,
    "3rd": 2278.0
  },
  "avg_size_of_commits": {
    "1st": 36.25,
    "median": 89.42857142857143,
    "3rd": 195.875
  },
  "mean_time_between_merged_prs": {
    "1st": 72.29378378378378,
    "median": 177.546568627451,
    "3rd": 432.7607953461758
  },
  "number_of_pull_requests_opened": {
    "1st": 7.0,
    "median": 18.0,
    "3rd": 38.0
  },
  "pulls_opened": {
    "1st": 7.0,
    "median": 18.0,
    "3rd": 38.0
  },
  "number_of_commits": {
    "1st": 8.75,
    "median": 20.5,
    "3rd": 68.0
  },
  "pulls_closed": {
    "1st": 8.5,
    "median": 31.0,
    "3rd": 93.0
  },
  "mean_discussion_duration": {
    "1st": 6.0,
    "median": 14.2,
    "3rd": 36.0
  },
  "mean_time_between_comments": {
    "1st": 27.333333333333332,
    "median": 58.0,
    "3rd": 119.5
  },
  "mean_words": {
    "1st": 58.25,
    "median": 133.5,
    

In [94]:
def get_group_quartile(dict_quartiles, metric_name, value_metric):
    if value_metric >= quartiles_metrics[metric_name]['3rd']:
        return "HIGH"
    elif value_metric <= quartiles_metrics[metric_name]['1st']:
        return "LOW"
    else:
        return "MEDIUM"

In [98]:
user_quartiles_metrics = {}
for user in users_metrics:
    username = user['username']
    user_quartiles_metrics[username] = {}
    for metric in metrics:
        if metric in user:
            user_quartiles_metrics[username][metric] = get_group_quartile(dict_quartiles=quartiles_metrics, 
                                                                      metric_name=metric, 
                                                                      value_metric=user[metric])
        else:
            user_quartiles_metrics[username][metric] = "NONE"

for user in refactorings_info:
    if user in user_quartiles_metrics:
        user_quartiles_metrics[user]['refactorings'] = get_group_quartile(dict_quartiles=quartiles_metrics, 
                                                                          metric_name='refactorings', 
                                                                          value_metric=refactorings_info[user]['num_refactorings'])
    else:
        user_quartiles_metrics[user] = {}
        user_quartiles_metrics[user]['refactorings'] = get_group_quartile(dict_quartiles=quartiles_metrics, 
                                                                          metric_name='refactorings', 
                                                                          value_metric=refactorings_info[user]['num_refactorings'])

In [100]:
json_formatted_str = json.dumps(user_quartiles_metrics, indent=2)

with open(f"data/metrics/groups_users_metrics_{system}.json", "w") as write_file:
    json.dump(user_quartiles_metrics, write_file, indent=4)